In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

/Users/yunsungsong/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
x_data = mnist.train.images
y_data = mnist.train.labels

In [3]:
x_data.shape

(55000, 784)

In [4]:
# CNN부분은 이미지를 전처리하고
# DNN에서 실제로 구현한다.

In [5]:
training_epoch = 15
batch_size = 100
learning_reat = 0.001

In [6]:
X = tf.placeholder(tf.float32, [None,784 ])
Y = tf.placeholder(tf.float32, [None,10 ])
# drop을 위한 placeholder
keep_prob = tf.placeholder(tf.float32)

In [7]:
# CNN에 넣으려면 xdata를 image 파일 형태로 reshape해줘야한다.
# 784개의 하나의 vector를 28*28*1 짜리 matrix로 변환
X_img = tf.reshape(X,shape = [-1,28,28,1])

In [8]:
# CNN 첫번째 층 필터 32

# weight 변수지정
W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
# convol (filter) layer
L1 = tf.nn.conv2d(X_img, W1, strides = [1,1,1,1], padding = "SAME")
# relu layer
L1 = tf.nn.relu(L1)
# dropout
L1 = tf.nn.dropout(L1, keep_prob = keep_prob)
# pooling(max)
L1 = tf.nn.max_pool(L1,ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

# -1 * 14 * 14 * 32

# CNN 두번째 층 필터 64

W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
# convol (filter) layer
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = "SAME")
# relu layer
L2 = tf.nn.relu(L2)
# dropout
L2 = tf.nn.dropout(L2, keep_prob = keep_prob)
# pooling(max)
L2 = tf.nn.max_pool(L2,ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

# -1 * 7 * 7 * 64

# CNN 세번째 층 필터 128

W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
# convol (filter) layer
L3 = tf.nn.conv2d(L2, W3, strides = [1,1,1,1], padding = "SAME")
# relu layer
L3 = tf.nn.relu(L3)
# dropout
L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
# pooling(max)
L3 = tf.nn.max_pool(L3,ksize = [1,2,2,1],strides=[1,2,2,1],padding="SAME")

# -1 * 4 * 4 * 128
# -1이 image갯수, 4 x 4 , depth가 128개

# 이거 DNN으로 넘기려면(결과값) 다시 하나의 vector로 reshape해줘야한다.
L3_flat = tf.reshape(L3, shape = [-1, 4 * 4 * 128])

# DNN 첫번째 층 node 625

W3 = tf.get_variable("W3",shape = [4 * 4 * 128, 625],initializer= tf.contrib.layers.xavier_initializer())
L4 = tf.nn.relu(tf.matmul(L3_flat,W3))
L4 = tf.nn.dropout(L4, keep_prob = keep_prob)

# DNN 두번쨰 층 node 256

W4 = tf.get_variable("W4",shape = [625, 256],initializer= tf.contrib.layers.xavier_initializer())
L5 = tf.nn.relu(tf.matmul(L4,W4))
L5 = tf.nn.dropout(L5, keep_prob = keep_prob)

# 마지막
W5 = tf.get_variable("W5",shape = [256,10], initializer= tf.contrib.layers.xavier_initializer())
logits = tf.matmul(L5,W5)

hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
# optimizer
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# 그래프그리기
train = optimizer.minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(training_epoch):
    total_batch = int(mnist.train.num_examples/batch_size)
    avg_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # slicing을 해서 x데이터와 y데이터를 나눠서 반환해준다.
        c, _ = sess.run([cost,train], feed_dict = {X:batch_xs,Y:batch_ys,keep_prob:0.7})
        avg_cost += c/total_batch
        # cost를 다 더해서 total_Batch로 나눈것과 같은 효과
    print("Epoch",epoch, "Cost",avg_cost )

Epoch 0 Cost 0.4988601893999359
Epoch 1 Cost 0.2811866782199251
Epoch 2 Cost 0.24630567378618481
Epoch 3 Cost 0.2470569847930564
Epoch 4 Cost 0.23878293129530817
Epoch 5 Cost 0.24203002585606162
Epoch 6 Cost 0.24807186138900852
Epoch 7 Cost 0.22316944968294022
Epoch 8 Cost 0.2503615116090935
Epoch 9 Cost 0.2527445917305621
Epoch 10 Cost 0.24977120459079769
Epoch 11 Cost 0.28587415093725377
Epoch 12 Cost 0.25805462069809426
Epoch 13 Cost 0.24398258692838945
Epoch 14 Cost 0.24182686544277457


In [11]:
x_test = mnist.test.images
y_test = mnist.test.labels

is_correct = tf.equal(tf.argmax(hypothesis,1),tf.argmax(Y,1))
# hypothesis(예측값) 중 가장 큰 값의 index를 뽑아내는 것
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
sess.run(accuracy, feed_dict={X:x_test,Y:y_test,keep_prob : 1})
# accuracy할때는 1을 넣어야한다.

0.9653